In [1]:
import pandas as pd 
import numpy as np

In [2]:
stores = pd.read_csv('1_stores_corrupted_cleaned.csv')
products = pd.read_csv('2_products_corrupted_cleaned.csv')
customers = pd.read_csv('3_customers_corrupted_cleaned.csv')
promotions = pd.read_csv('4_promotions_corrupted_cleaned.csv')
sales_header = pd.read_csv('6_sales_headers_corrupted_cleaned.csv')
sales_line_items = pd.read_csv('7_sales_line_items_corrupted_cleaned.csv')

In [4]:


#Ensure transaction date column is treated as datetime 
sales_header["transaction_date"] = pd.to_datetime(sales_header["transaction_date"], errors="coerce")

# Filter sales from last year (adjust threshold based on dataset date range)
filtered_sales = sales_header[sales_header["transaction_date"] >= "2024-01-01"]

#  Customer purchase history 
customer_history = filtered_sales.groupby("customer_id").agg(
    total_spend_last_year=("total_amount", "sum"),
    purchase_frequency=("transaction_id", "count"),
    avg_order_value=("total_amount", "mean")
).reset_index()

#  Create loyalty weight mapping
loyalty_mapping = {
    "basic": 1.0,
    "silver": 1.1,
    "gold": 1.25,
    "platinum": 1.5
}

customers["loyalty_status"] = customers["loyalty_status"].str.lower()
customers["loyalty_multiplier"] = customers["loyalty_status"].map(loyalty_mapping).fillna(1.0)

# Merge loyalty weight with purchase history 
result = customer_history.merge(customers[["customer_id", "loyalty_multiplier"]], on="customer_id", how="left")

#  Estimate future spend
result["estimated_future_spend"] = (result["avg_order_value"] * 12 * result["loyalty_multiplier"]).round(2)

# Sort results like SQL ORDER BY 
result = result.sort_values(by="estimated_future_spend", ascending=False)

result.head(10)

,customer_id,total_spend_last_year,purchase_frequency,avg_order_value,loyalty_multiplier,estimated_future_spend
81,C0126,341548,2,170774.0,1.10,2254216.8
115,C0190,159446,1,159446.0,1.10,2104687.2
61,C0093,154834,1,154834.0,1.10,2043808.8
3,C0005,157254,1,157254.0,1.00,1887048.0
30,C0045,157033,1,157033.0,1.00,1884396.0
108,C0176,220251,2,110125.5,1.25,1651882.5
96,C0153,270033,2,135016.5,1.00,1620198.0
80,C0125,133152,1,133152.0,1.00,1597824.0
122,C0199,130643,1,130643.0,1.00,1567716.0
118,C0194,124492,1,124492.0,1.00,1493904.0


In [5]:
customer_features = filtered_sales.groupby("customer_id").agg(
    total_spend=("total_amount", "sum"),
    purchase_frequency=("transaction_id", "count"),
    avg_order_value=("total_amount", "mean"),
    last_purchase_date=("transaction_date", "max")
).reset_index()

# Recency (how long since last purchase)
customer_features["recency_days"] = (pd.to_datetime("today") - customer_features["last_purchase_date"]).dt.days
customer_features.drop(columns=["last_purchase_date"], inplace=True)

In [6]:
# Map loyalty values
loyalty_map = {"basic":0, "silver":1, "gold":2, "platinum":3}
customers["loyalty_status"] = customers["loyalty_status"].str.lower()
customers["loyalty_level"] = customers["loyalty_status"].map(loyalty_map).fillna(0)

# Merge with features
model_df = customer_features.merge(customers[["customer_id","loyalty_level"]], on="customer_id", how="left")
model_df = model_df.fillna(0)

In [8]:
#future_spend_target = total_spend * growth factor based on loyalty

In [9]:
loyalty_multiplier = {0:1.0, 1:1.1, 2:1.25, 3:1.5}

model_df["future_spend"] = model_df.apply(lambda x: x.total_spend * loyalty_multiplier[x.loyalty_level], axis=1)

In [10]:
from sklearn.model_selection import train_test_split

X = model_df[["total_spend","purchase_frequency","avg_order_value","recency_days","loyalty_level"]]
y = model_df["future_spend"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

,n_estimators,200
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [12]:
from sklearn.metrics import mean_absolute_error, r2_score

predictions = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, predictions))
print("R2 Score:", r2_score(y_test, predictions))

MAE: 6081.2687099999985
R2 Score: 0.9286493124263373


In [13]:
model_df["predicted_future_spend"] = model.predict(X)

model_df.sort_values("predicted_future_spend", ascending=False).head(10)

,customer_id,total_spend,purchase_frequency,avg_order_value,recency_days,loyalty_level,future_spend,predicted_future_spend
81,C0126,341548,2,170774.000000,367,1.0,375702.80,329503.96900
96,C0153,270033,2,135016.500000,367,0.0,270033.00,269607.84650
108,C0176,220251,2,110125.500000,367,2.0,275313.75,257073.74600
42,C0065,217717,4,54429.250000,368,0.0,217717.00,240195.72775
60,C0092,219430,4,54857.500000,368,1.0,241373.00,239423.35825
45,C0069,193208,3,64402.666667,367,0.0,193208.00,194035.98025
7,C0010,170976,4,42744.000000,368,0.0,170976.00,169838.99050
115,C0190,159446,1,159446.000000,369,1.0,175390.60,169392.46800
51,C0077,154048,2,77024.000000,370,1.0,169452.80,168627.78700
61,C0093,154834,1,154834.000000,368,1.0,170317.40,168359.45600


In [14]:
def predict_future_spend(total_spend, purchase_frequency, avg_order_value, recency_days, loyalty_status):
    
    # Map loyalty to numeric
    loyalty_map = {"basic":0, "silver":1, "gold":2, "platinum":3}
    loyalty_level = loyalty_map.get(loyalty_status.lower(), 0)
    
    # Prepare input for model
    input_data = np.array([[total_spend, purchase_frequency, avg_order_value, recency_days, loyalty_level]])
    
    # Predict using trained model
    predicted_value = model.predict(input_data)[0]
    
    return round(predicted_value, 2)

In [15]:
example_prediction = predict_future_spend(
    total_spend=15000,
    purchase_frequency=12,
    avg_order_value=1250,
    recency_days=10,
    loyalty_status="Gold"
)


example_prediction

C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


np.float64(13935.77)

In [16]:
import pickle

# save the trained model to file
with open("future_spend_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model saved successfully!")

Model saved successfully!
